In [3]:
from pybrain.datasets.supervised import SupervisedDataSet
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer, RPropMinusTrainer
from pybrain.structure import  LinearLayer,SigmoidLayer

import PIL.Image
import PIL.ImageStat
import Xlib.display

import numpy as np

import os

In [4]:
def read_image(image_path):
    return np.asarray(PIL.Image.open(image_path))

def get_histogram(image_array):
    result = np.array([0.0 for _ in xrange(256)])
    for x in image_array:
        result[x] += 1.0
    return result

def get_histograms(image):
    w, h, d = image.shape
    image_flat = image.reshape(w * h, d)
    histogram_r = get_histogram(image_flat[:, 0])
    histogram_g = get_histogram(image_flat[:, 1])
    histogram_b = get_histogram(image_flat[:, 2])

    histogram_r_norm = (histogram_r / float(sum(histogram_r))).reshape(256, 1)
    histogram_g_norm = (histogram_g / float(sum(histogram_g))).reshape(256, 1)
    histogram_b_norm = (histogram_b / float(sum(histogram_b))).reshape(256, 1)

    return np.concatenate((histogram_r_norm, histogram_g_norm, histogram_b_norm), axis=0).reshape(768)



In [3]:
n_input = 256 * 3
n_hidden = 256 * 3
n_out = 1

network = buildNetwork(n_input, n_hidden, n_hidden, n_out, outclass=SigmoidLayer)
network.sortModules()

In [4]:
training_set = SupervisedDataSet(n_input, n_out)

In [5]:
base_dir = './day_or_night/day/'
images_paths = os.listdir(base_dir)
class_type = 1
for image_path in images_paths:
    training_set.addSample(get_histograms(read_image(base_dir + image_path)), class_type)
    print image_path + " added to day set."
print "DONE"

73.jpg added to day set.
24.jpg added to day set.
58.jpg added to day set.
53.jpg added to day set.
42.jpg added to day set.
18.jpg added to day set.
48.jpg added to day set.
70.jpg added to day set.
23.jpg added to day set.
64.jpg added to day set.
61.jpg added to day set.
3.jpg added to day set.
56.jpg added to day set.
74.jpg added to day set.
22.jpg added to day set.
60.jpg added to day set.
81.jpg added to day set.
54.jpg added to day set.
11.jpg added to day set.
5.jpg added to day set.
35.jpg added to day set.
89.jpg added to day set.
90.jpg added to day set.
66.jpg added to day set.
41.jpg added to day set.
4.jpg added to day set.
76.jpg added to day set.
15.jpg added to day set.
29.jpg added to day set.
13.jpg added to day set.
85.jpg added to day set.
78.jpg added to day set.
47.jpg added to day set.
75.jpg added to day set.
31.jpg added to day set.
9.jpg added to day set.
93.jpg added to day set.
98.jpg added to day set.
37.jpg added to day set.
1.jpg added to day set.
63.jp

In [6]:
base_dir = './day_or_night/night/'
images_paths = os.listdir(base_dir)
class_type = 0
for image_path in images_paths:
    training_set.addSample(get_histograms(read_image(base_dir + image_path)), class_type)
    print image_path + " added to night set."
print "DONE"

73.jpg added to night set.
24.jpg added to night set.
13.jpe added to night set.
58.jpg added to night set.
53.jpg added to night set.
42.jpg added to night set.
18.jpg added to night set.
48.jpg added to night set.
29.JPG added to night set.
70.jpg added to night set.
23.jpg added to night set.
99.jpg added to night set.
64.jpg added to night set.
61.jpg added to night set.
3.jpg added to night set.
56.jpg added to night set.
74.jpg added to night set.
22.jpg added to night set.
60.jpg added to night set.
81.jpg added to night set.
54.jpg added to night set.
11.jpg added to night set.
5.jpg added to night set.
35.jpg added to night set.
89.jpg added to night set.
90.jpg added to night set.
66.jpg added to night set.
41.jpg added to night set.
4.jpg added to night set.
76.jpg added to night set.
15.jpg added to night set.
13.jpg added to night set.
85.jpg added to night set.
78.jpg added to night set.
47.jpg added to night set.
75.jpg added to night set.
31.jpg added to night set.
9.jp

In [11]:
trainer = BackpropTrainer(network, training_set, learningrate=0.001, momentum=0.8, verbose=True)
for _ in range(10):
    trainer.trainEpochs(2)
print "DONE"

Total error: 0.248748221735
Total error: 0.248747508923
Total error: 0.248746655898
Total error: 0.248745620024
Total error: 0.248744322968
Total error: 0.248742675729
Total error: 0.248740512861
Total error: 0.248737513304
Total error: 0.248733071818
Total error: 0.248725934628
Total error: 0.24871263993
Total error: 0.248677045621
Total error: 0.247648282662
Total error: 0.133574168548
Total error: 0.0667829448229
Total error: 0.0375363512066
Total error: 0.0339271079221
Total error: 0.0315121124064
Total error: 0.029395919515
Total error: 0.0263875593971
DONE


In [14]:
def parse(x):
    if x[0] > 0.5:
        return 1
    else:
        return 0

In [10]:
path = './day_or_night/test/day2.jpg'
print network.activate(get_histograms(read_image(path)))

[ 0.97116928]


In [6]:
from pybrain.tools.xml.networkwriter import NetworkWriter
NetworkWriter.writeToFile(network, 'network.xml')

In [7]:
from pybrain.tools.xml.networkreader import NetworkReader
network = NetworkReader.readFrom('network.xml') 

In [22]:
tp=0
fp=0
fn=0
tn=0
base_dir = './day_or_night/day/'
images_paths = os.listdir(base_dir)
counter = 0
all_count = len (images_paths)
for image_path in images_paths:
    result = parse(network.activate(get_histograms(read_image(base_dir+image_path))))
    if result == 1:
        tp += 1
    else:
        fn += 1
    counter += 1
    print counter, all_count
        
base_dir = './day_or_night/night/'
images_paths = os.listdir(base_dir)
counter = 0
all_count = len (images_paths)
for image_path in images_paths:
    result = parse(network.activate(get_histograms(read_image(base_dir+image_path))))
    if result == 0:
        tn += 1
    else:
        fp += 1
    counter += 1
    print counter, all_count
print tp, fp, fn, tn

1 100
2 100
3 100
4 100
5 100
6 100
7 100
8 100
9 100
10 100
11 100
12 100
13 100
14 100
15 100
16 100
17 100
18 100
19 100
20 100
21 100
22 100
23 100
24 100
25 100
26 100
27 100
28 100
29 100
30 100
31 100
32 100
33 100
34 100
35 100
36 100
37 100
38 100
39 100
40 100
41 100
42 100
43 100
44 100
45 100
46 100
47 100
48 100
49 100
50 100
51 100
52 100
53 100
54 100
55 100
56 100
57 100
58 100
59 100
60 100
61 100
62 100
63 100
64 100
65 100
66 100
67 100
68 100
69 100
70 100
71 100
72 100
73 100
74 100
75 100
76 100
77 100
78 100
79 100
80 100
81 100
82 100
83 100
84 100
85 100
86 100
87 100
88 100
89 100
90 100
91 100
92 100
93 100
94 100
95 100
96 100
97 100
98 100
99 100
100 100
1 101
2 101
3 101
4 101
5 101
6 101
7 101
8 101
9 101
10 101
11 101
12 101
13 101
14 101
15 101
16 101
17 101
18 101
19 101
20 101
21 101
22 101
23 101
24 101
25 101
26 101
27 101
28 101
29 101
30 101
31 101
32 101
33 101
34 101
35 101
36 101
37 101
38 101
39 101
40 101
41 101
42 101
43 101
44 101
45 101
46

In [23]:
precision=float(tp)/(float(tp)+float(fp))
recall=float(tp)/(float(tp)+float(fn))
print "precision", precision
print "recall", recall

precision 0.988888888889
recall 0.89
